In [97]:
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

# pip install numpy
# conda install -c anaconda pandas
# conda install -c conda-forge matplotlib

import datetime as dt # For defining dates

import time

# In Powershell Prompt : conda install -c conda-forge multitasking
# pip install -i https://pypi.anaconda.org/ranaroussi/simple yfinance

import yfinance as yf

# To show all your output File -> Preferences -> Settings Search for Notebook
# Notebook Output Text Line Limit and set to 100

# Used for file handling like deleting files
import os

# conda install -c conda-forge cufflinks-py
# conda install -c plotly plotly
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

from plotly.subplots import make_subplots

# New Imports
# Used to get data from a directory
import os
from os import listdir
from os.path import isfile, join

import warnings
warnings.simplefilter("ignore")



# constants

In [98]:
PATH = r"/Users/sammurray/Documents/stonks/"

S_DATE = "2017-02-01"
E_DATE = "2022-12-06"
S_DATE_DT = pd.to_datetime(S_DATE)
E_DATE_DT = pd.to_datetime(E_DATE)

In [99]:
def get_stock_df_from_csv(ticker):
    try:
        df = pd.read_csv(PATH + ticker + ".csv", index_col=0)
    except FileNotFoundError:
        print("File is not here" )
        print('Expected file: ' + PATH + ticker + ".csv")
    else: 
        return df 

In [100]:
files = [x for x in os.listdir(PATH) if os.path.isfile(os.path.join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]
# tickers.remove('.ds_Store')

tickers.sort()
len(tickers)


3262

# Add daily returns

In [101]:
def add_daily_return_to_df(df):
    df['daily_return'] = (df['Close'] / df['Close'].shift(1)) - 1
    # Save data to a CSV file
    # df.to_csv(PATH + ticker + '.csv')
    return df

# add cumulative return

In [102]:
def add_cum_return_to_df(df):
    df['cum_return'] = (1 + df['daily_return']).cumprod()
    # df.to_csv(PATH + ticker + '.csv')
    return df


# Add Bolinger Bands

In [103]:
def add_bollinger_bands(df):
    df['middle_band'] = df['Close'].rolling(window=20).mean()
    df['upper_band'] = df['middle_band'] + 1.96 * df['Close'].rolling(window=20).std()
    df['lower_band'] = df['middle_band'] - 1.96 * df['Close'].rolling(window=20).std()
    # df.to_csv(PATH + ticker + '.csv')
    return df

# Add ichimoku data to data frame (dope technical analysis tool)

In [104]:
def add_ichimoku(df):
    # conversion line (9 sessions)
    hi_val = df['High'].rolling(window=9).max()
    low_val = df['Low'].rolling(window=9).min()
    df['conversion'] = (hi_val + low_val) / 2

    # baseline (26 sessions)
    hi_val2 = df['High'].rolling(window=26).max()
    low_val2 = df['Low'].rolling(window=26).min()
    df['baseline'] = (hi_val2 + low_val2) / 2

    # Span A (26 sessions)
    df['SpanA'] = ((df['conversion'] + df['baseline']) /2)

    # Span B (52 sessions)
    hi_val3 = df['High'].rolling(window=52).max()
    low_val3 = df['Low'].rolling(window=52).min()

    df['SpanB'] = ((hi_val3 + low_val3) / 2).shift(26)

    # Lagging Span
    df['Lagging'] = df['Close'].shift(26)
    return df

# Add daily cumulative Bollinger bands and Ichimoku data frames

In [105]:
#for x in tickers:
#    try:
 #      print("working on :", x)
  #      new_df = get_stock_df_from_csv(x)
   #     new_df = add_daily_return_to_df(new_df)
    #    new_df = add_cum_return_to_df(new_df)
     #   new_df = add_bollinger_bands(new_df)
      #  new_df = add_Ichimoku(new_df)
       # new_df.to_csv(PATH + 'A' + '.csv')
    #except Exception as ex:
        #print(ex)
        # print(f'{count}/{len(tickers)}')



In [106]:
def plot_with_boll_bands(df, ticker):
    fig = go.Figure()

    candle = go.Candlestick(x=df.index, open=df['Open'], high=df['High'], 
    low=df['Low'], close=df['Close'], name='Candlestick')

    upper_line = go.Scatter(x=df.index, y=df['upper_band'], 
    line=dict(color='rgba(250, 0, 0, 0.75)', 
    width=1), name='Upper Band')

    mid_line = go.Scatter(x=df.index, y=df['middle_band'], 
    line=dict(color='rgba(0, 0, 250, 0.75)', 
    width=1), name='Middle Band')

    lower_line = go.Scatter(x=df.index, y=df['lower_band'], 
    line=dict(color='rgba(0, 250, 0, 0.75)', 
    width=1), name='Lower Band')

    fig.add_trace(candle)
    fig.add_trace(upper_line)
    fig.add_trace(mid_line)
    fig.add_trace(lower_line)

    fig.update_xaxes(title="Date", rangeslider_visible=True)
    fig.update_yaxes(title="Price")

    fig.update_layout(title=ticker + " Bollinger Bands", 
    height=800, width=1700, showlegend=True)
    fig.show()

# Plot bollinger

In [107]:
test_df = get_stock_df_from_csv("AMD")
plot_with_boll_bands(test_df, "AMD")

KeyError: 'upper_band'